In [70]:
import numpy as np
import numpy.random as npr
from scipy.stats import norm
from math import log, sqrt, pi, exp
import pandas as pd

## Задание 2

Find the price of a derivative which pays off $𝑆_T^5 − K$ if $𝑆_T > K$ and zero otherwise (you
may want to use Itô’s lemma for this).
Write a Python code to price this derivative and compare your results.

### Из рассчетов:

$$
F = S_0^5e^{10\sigma^2 T + 4rT}N(d_1) - Ke^{-rT}N(d_2),
$$
где
$$
d_1 = \frac{
  ln( \frac{S_0}{K} ) + (r + \frac{9}{2}\sigma^2)T}{
    \sigma \sqrt{T}
    }, \;\; d_2 = \frac{
  ln(\frac{S_0}{K}) + (r - \frac{\sigma^2}{2})T}{
    \sigma \sqrt{T}}.
$$

In [61]:
def d1(S,K,T,r,sigma):
    return (log(S/K) + (r + 9 * sigma**2/2.) * T) / (sigma * sqrt(T))

def d2(S,K,T,r,sigma):
    return (log(S/K) + (r - sigma**2/2.) * T) / (sigma * sqrt(T))

def deriv(S,K,T,r,sigma):
    price = S**5 * np.exp(10* sigma**2 * T) * np.exp(4 * r * T) * norm.cdf(d1(S,K,T,r,sigma)) - K * np.exp(-r*T) * norm.cdf(d2(S,K,T,r,sigma))
    return price

### Монте-Карло:

In [62]:
'''
param S0: изначальная цена актива
param K: страйк
param T: время экспирации
param r: безрисковая ставка
param sigma: волатильность актива
'''

def simulate_stock_price(S0, r, sigma, T, n_sims):
    WT = np.sqrt(T)*npr.normal(0,1,(n_sims,1))
    ST = S0 * np.exp(sigma * WT + (r - 0.5 * sigma ** 2) * T)
    return ST

def price_derivative(S0, K, T, r, sigma, n_sims):
    ST = simulate_stock_price(S0, r, sigma, T, n_sims)
    payoffs = np.where(ST > K, ST**5 - K, 0)
    price_disc = np.exp(-r * T) * np.mean(payoffs)
    return price_disc

### Сравнение:

In [64]:
def print_result(S0, K, T, r, sigma, n_sims):
    price_disc = price_derivative(S0, K, T, r, sigma, n_sims)
    exponent_disc = len(str(int(price_disc))) - 1
    mantissa_disc = price_disc / (10 ** exponent_disc)

    price = deriv(S0, K, T, r, sigma)
    exponent = len(str(int(price))) - 1
    mantissa = price / (10 ** exponent)

    print(f"Simulated price of the derivative is: {mantissa_disc:.2f} * 10^{exponent_disc}")
    print(f"Theoretical price of the derivative is: {mantissa:.2f} * 10^{exponent}")

In [65]:
S0 = 100
K = 150
T = 1
r = 0.05
sigma = 0.1
n_simulations = 10000000

print_result(S0, K, T, r, sigma, n_simulations)

Simulated price of the derivative is: 1.29 * 10^7
Theoretical price of the derivative is: 1.29 * 10^7


In [68]:
S0 = 100
K = 150
T = 1
r = 0.05
sigma = 0.5
n_simulations = 10000000

print_result(S0, K, T, r, sigma, n_simulations)

Simulated price of the derivative is: 1.42 * 10^11
Theoretical price of the derivative is: 1.40 * 10^11


Сходится!!

## Задание 3

Pick your favorite stock and find the prices of at-the-money calls and puts on it using
Black-Scholes-Merton model.

В этом задании буду работать с Roblox Stock: рассчитаю стоимости годовых at-the-money put- и call- опционов на момент 2.12.2024 (первый рабочий день декабря).

Волатильность рассчитаем из годовых исторических данных ноябрь 2023 - ноябрь 2024. Для оценки risk free rate возьмем среднее значение 1 Year Treasury Rate за рассматриваемый год.

In [76]:
df = pd.read_csv('/content/Roblox Stock Price History.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)
df.head()

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2024-12-02,52.16,50.99,52.65,50.94,8.23M,4.05%
2024-11-29,50.13,49.81,50.73,49.68,3.63M,1.54%
2024-11-27,49.37,49.09,50.54,48.61,5.62M,0.63%
2024-11-26,49.06,49.18,50.01,48.93,6.92M,-0.20%
2024-11-25,49.16,49.13,49.49,47.95,9.52M,-0.45%


In [92]:
trr = pd.read_csv('/content/United States 1-Year Bond Yield Historical Data.csv')
trr['Date'] = pd.to_datetime(trr['Date'])
trr.set_index('Date', inplace = True)
trr.head()

,Price,Open,High,Low,Change %
Date,,,,,
2024-11-29,4.291,4.345,4.371,4.300,-0.90%
2024-11-27,4.330,4.372,4.372,4.323,-0.85%
2024-11-26,4.367,4.391,4.413,4.367,-0.10%
2024-11-25,4.371,4.438,4.440,4.378,-0.89%
2024-11-22,4.411,4.425,4.430,4.378,0.46%


In [88]:
sigma = df.Price.std()
r = trr.Price.mean()
S = df.iloc[0, 0]
K = S
T = 1

In [89]:
def d1(S, K, T, r, sigma):
    return(log(S/K) + (r + sigma**2/2.)*T) / (sigma * sqrt(T))

def d2(S, K, T, r, sigma):
    return d1(S, K, T, r, sigma) - sigma * sqrt(T)

def bs_call(S, K, T, r, sigma):
    price = S * norm.cdf(d1(S, K, T, r, sigma))-K * np.exp(-r*T) * norm.cdf(d2(S, K, T, r, sigma))
    return price

def bs_put(S, K, T, r, sigma):
    price = K * np.exp(-r*T) * norm.cdf(-1 * d2(S, K, T, r, sigma)) - S * norm.cdf(-1 * d1(S, K, T, r, sigma))
    return price

In [90]:
bs_call(S, K, T, r, sigma)

52.10234647114477

In [91]:
bs_put(S, K, T, r, sigma)

0.376859114430985